# How to use `unfold`

Author: [Romain Sacchi](romain.sacchi@psi.ch)

Github repo: https://github.com/romainsacchi/

## fold
Several `brightway` databases are "folded" in a data package.

In [ ]:
from unfold import Fold
import bw2data
bw2data.projects.set_current("carculator") # <-- name of the brightway project containing both the source database and the databases to fold

In [ ]:
f = Fold()

In [ ]:
f.fold()

In [ ]:
# alternatively, you can answer these questions directly by passing the following arguments
f.fold(
    package_name="a package name",
    package_description="some description",
    source="cutoff38",
    system_model="cut off",
    version="3.6",
    databases_to_fold=["ecoinvent_image_SSP2-Base_2025", "ecoinvent_image_SSP2-Base_2050"],
    descriptions=["some db", "some other db"],
)

## unfold
Here, we read a data package, and we unfold and write the folded databases back into a `brightway` project.
It will write one database per scenario to your `brightway` project.

In [ ]:
from unfold import Unfold
import bw2data
bw2data.projects.set_current("carculator")

In [ ]:
u = Unfold("srgd.zip")

In [ ]:
u.unfold()

In [ ]:
# alternatively, you can answer these questions directly by passing the following arguments
u.unfold(
    dependencies={
        "ecoinvent 3.6 cutoff": "ecoinvent 3.6 cutoff",
        "biosphere3": "biosphere3"
    }
)

# unfold
## superstructure databases
`unfold` can also unfold [superstructure](https://link.springer.com/article/10.1007/s11367-021-01974-2) databases, to be used with [Activity Browser](https://github.com/LCA-ActivityBrowser/activity-browser).
It will write only one database, regardless of the number of scenarios.
It will also export a scenario difference spreadsheet to be read by Activity Browser.

In [ ]:
u.unfold(
    scenarios=[0, 1],
    dependencies={
        "cutoff38": "cutoff38",
        "biosphere3": "biosphere3"
    },
    superstructure=True # <-- NEW
)